<a href="https://colab.research.google.com/github/assaflv/final_project/blob/combine_algo/ML_Project_Improvment_factor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [303]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingClassifier 
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import lightgbm

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [304]:
import pandas as pd
import numpy as np
WizWhy_file = "WizWhy02.txt"
df = pd.read_csv("churn.csv")
print(df.shape)
df.head()

(10000, 14)


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [305]:
df_wizwhy = pd.read_csv(WizWhy_file)
df_wizwhy['Prediction'].replace({' No 1.00':0,'1.00':1},inplace= True)
y_wiz = df_wizwhy['Prediction'].to_numpy()
print(y_wiz)

[0 0 1 ... 0 0 0]


In [306]:
percentage_one = (df[df.Exited == 1].sum().Exited)/df.shape[0] #The percentage of labels that are 1
percentage_zero  = (df.shape[0] - percentage_one*df.shape[0])/df.shape[0] #The percentage of labels that are 0
print("The percentage of labels that are 1:",percentage_one,"\n","The percentage of labels that are 0:",percentage_zero)

The percentage of labels that are 1: 0.2037 
 The percentage of labels that are 0: 0.7963


In [307]:
X = df[['Geography','Gender','CreditScore','Age','Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember','EstimatedSalary']]
print(X.shape)
X.head()

(10000, 10)


,Geography,Gender,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,France,Female,619,42,2,0.00,1,1,1,101348.88
1,Spain,Female,608,41,1,83807.86,1,0,1,112542.58
2,France,Female,502,42,8,159660.80,3,1,0,113931.57
3,France,Female,699,39,1,0.00,2,0,0,93826.63
4,Spain,Female,850,43,2,125510.82,1,1,1,79084.10


In [308]:
def make_dummy_df(X,category_list):
  return pd.get_dummies(X, columns=category_list)







Creating features:

In [309]:
X_unDummy = X.copy()
X_dummy = make_dummy_df(X,["Geography","Gender"])
y= df.Exited


Split Date:

In [310]:
def split_data(X,y,test_size):
  X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=percentage_one, random_state=42)  
  #print(" The percentage of labels in the train set are 1:",(y_train.sum())/y_train.shape[0],"\n","The percentage of labels in the test set are 1:",(y_test.sum())/y_test.shape[0])
  return X_train, X_test, y_train, y_test

In [311]:
X_d_train, X_d_test,y_d_train,y_d_test = split_data(X_dummy,y,percentage_one)
X_ud_train, X_ud_test,y_ud_train,y_ud_test = split_data(X_unDummy,y,percentage_one)

In [312]:
def AVG_cost(prediction_vector, y_test,c1 = 20,c2= 79):#c1 - the cost of fp and c2 the cost of fn
  from sklearn.metrics import confusion_matrix
  #print(Model.fit(X_train, y_train).predict(X_test))
  tn, fp, fn, tp = confusion_matrix(y_test,prediction_vector ).ravel()
  return (fp*c1 + fn*c2)/(prediction_vector.shape[0])

In [313]:
def EAVG_cost( y_test,c1 = 20,c2= 79):#c1 - the cost of fp and c2 the cost of fn
  all_one = np.array([1]*y_test.shape[0])
  all_zero = np.array([0]*y_test.shape[0])
  return min(AVG_cost(all_one, y_test,c1,c2),AVG_cost(all_zero, y_test,c1,c2))

In [314]:
def improvment_factor(prediction_vector, y_test,c1 = 20,c2= 79):
  return EAVG_cost( y_test,c1 ,c2)/AVG_cost(prediction_vector, y_test,c1 ,c2)

In [315]:
def combine_algo(pred_vectors,W_vectors):
  combine_result_y = []
  #print('pred_vectors', pred_vectors)
  pred_len = pred_vectors[1].shape[0]

  vectors_df = pd.DataFrame(pred_vectors)
  for col in range(0,pred_len):
    count_res = pd.value_counts(vectors_df[col])
    max_value = (vectors_df[col].value_counts(sort=True)[:1].index.tolist())[0]
    combine_result_y.append(max_value)

  #print(np.array(combine_result_y).shape[0])
  return(np.array(combine_result_y))

In [316]:
def pred_vector(model,X_train,y_train,X_test):
  return model.fit(X_train, y_train).predict(X_test)

def prediction_process(X_train,y_train,X_test,y_wiz):
  ada_boost = AdaBoostClassifier()
  lgmc = LGBMClassifier()

  algo_model_list= [ada_boost,lgmc]

  pred_vectors = []
  W = []
  for model in algo_model_list:
    pred_vectors.append(pred_vector(model,X_train,y_train,X_test))
    W.append(1)

  pred_vectors.append(y_wiz)
  return(combine_algo(pred_vectors,W))


In [317]:
def Grade(model,X_train,X_test, y_train, y_test):
  return improvment_factor(pred_vector(model,X_train, y_train, X_train),y_train),improvment_factor(pred_vector(model,X_train, y_train, X_test),y_test)

In [318]:
train_dic = {}
test_dic = {}

 Model 1: Nearest neighbors

In [319]:
for i in range(1,10,2):
  improvment_train,improvment_test = Grade(KNeighborsClassifier(n_neighbors=i),X_d_train, X_d_test,y_d_train,y_d_test)
  print("for ",i," Neighbors- or train is",improvment_train," and for test is ",improvment_test)

improvment_train,improvment_test = Grade(KNeighborsClassifier(n_neighbors=5),X_d_train, X_d_test,y_d_train,y_d_test)
train_dic['KN'] = improvment_train
test_dic['KN'] = improvment_test

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  


for  1  Neighbors- or train is inf  and for test is  1.0012234526461086
for  3  Neighbors- or train is 1.5170397354167666  and for test is  1.0174375976282317
for  5  Neighbors- or train is 1.2152977767538302  and for test is  1.0211812887950342
for  7  Neighbors- or train is 1.1121262166625672  and for test is  1.0056401046097614
for  9  Neighbors- or train is 1.0643843217702746  and for test is  1.0064328960645812


Model 2: Decision Tree

In [320]:
improvment_train,improvment_test = Grade(DecisionTreeRegressor(),X_d_train, X_d_test,y_d_train,y_d_test)
print("Grades: train is",improvment_train," and for test is ",improvment_test)
train_dic['DC'] = improvment_train
test_dic['DC'] = improvment_test

Grades: train is inf  and for test is  1.5307434052757793


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  


Model 3: Random forest

In [321]:
# initialize a random forest classifier with 1000 decision trees
model_RF = RandomForestClassifier(n_estimators = 1000, random_state = 1) 
# random_state is an arbitrary seed to the random number generator 
improvment_train,improvment_test = Grade(model_RF,X_d_train, X_d_test,y_d_train,y_d_test)
print("Grades: train is",improvment_train," and for test is ",improvment_test)
train_dic['RF']= improvment_train
test_dic['RF'] = improvment_test

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  


Grades: train is inf  and for test is  1.7233261339092873


Model 4: GradientBoosting

In [322]:

model_GB = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0)
improvment_train,improvment_test = Grade(model_GB,X_d_train, X_d_test,y_d_train,y_d_test)
print("Grades: train is",improvment_train," and for test is ",improvment_test)
train_dic['GB'] = improvment_train
test_dic['GB'] = improvment_test

Grades: train is 1.7707036658880793  and for test is  1.7508365790772944


Model 5: AdaBoostClassifier

In [323]:
improvment_train,improvment_test = Grade(AdaBoostClassifier(),X_d_train, X_d_test,y_d_train,y_d_test)
print("Grades: train is",improvment_train," and for test is ",improvment_test)

Grades: train is 1.7421458943978863  and for test is  1.7738995108937303


Model 6: LGBMClassifier

In [324]:
improvment_train,improvment_test = Grade(LGBMClassifier(),X_d_train, X_d_test,y_d_train,y_d_test)
print("Grades: train is",improvment_train," and for test is ",improvment_test)
train_dic['LGB'] = improvment_train
test_dic['LGB'] = improvment_test

Grades: train is 2.660502259115267  and for test is  1.83858517195691


Model 7: XGBClassifier

In [325]:
improvment_train,improvment_test = Grade(XGBClassifier(),X_d_train, X_d_test,y_d_train,y_d_test)
print("Grades: train is",improvment_train," and for test is ",improvment_test)
train_dic['XGB'] = improvment_train
test_dic['XGB'] = improvment_test

Grades: train is 1.8004437112463734  and for test is  1.7177610333692142


Auxiliary function:

when the label ('Exited') is 1 it's mean the custmer left us.
FP = we think its 1 and its 0
FN = we think its 0 and its 1


Combine algo:

In [326]:
combine_y_test = prediction_process(X_d_train,y_d_train,X_d_test,y_wiz)
combine_y_train = prediction_process(X_d_train,y_d_train,X_d_train,y_wiz)

improvment_train, improvment_test = improvment_factor(combine_y_train,y_train),improvment_factor(combine_y_test,y_test)
print("Grades: train is",improvment_train," and for test is ",improvment_test)
train_dic['comb'] = improvment_train
test_dic['comb'] = improvment_test

Grades: train is 2.492763896272668  and for test is  1.9655129942111096


In [327]:
print(train_dic)
print(test_dic)

{'KN': 1.2152977767538302, 'DC': inf, 'RF': inf, 'GB': 1.7707036658880793, 'LGB': 2.660502259115267, 'XGB': 1.8004437112463734, 'comb': 2.492763896272668}
{'KN': 1.0211812887950342, 'DC': 1.5307434052757793, 'RF': 1.7233261339092873, 'GB': 1.7508365790772944, 'LGB': 1.83858517195691, 'XGB': 1.7177610333692142, 'comb': 1.9655129942111096}
